In [1]:
import pandas as pd
from pathlib import Path
from efficientnet.config import Config
import json

/home/ubuntu/.local/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
Config.image_directory

PosixPath('/home/ubuntu/data/images/output')

In [3]:
train_images = list(Config.image_directory.glob("*.png"))
train_data = "/home/ubuntu/data/code/efficientnet-detector/train.csv"

In [4]:
train_images

[PosixPath('/home/ubuntu/data/images/output/10006_1459541791.png'),
 PosixPath('/home/ubuntu/data/images/output/10006_1864590858.png'),
 PosixPath('/home/ubuntu/data/images/output/10006_1874946579.png'),
 PosixPath('/home/ubuntu/data/images/output/10006_462822612.png'),
 PosixPath('/home/ubuntu/data/images/output/10011_1031443799.png'),
 PosixPath('/home/ubuntu/data/images/output/10011_220375232.png'),
 PosixPath('/home/ubuntu/data/images/output/10011_270344397.png'),
 PosixPath('/home/ubuntu/data/images/output/10011_541722628.png'),
 PosixPath('/home/ubuntu/data/images/output/10025_1365269360.png'),
 PosixPath('/home/ubuntu/data/images/output/10025_288394860.png'),
 PosixPath('/home/ubuntu/data/images/output/10025_562340703.png'),
 PosixPath('/home/ubuntu/data/images/output/10025_893612858.png'),
 PosixPath('/home/ubuntu/data/images/output/10038_1350492010.png'),
 PosixPath('/home/ubuntu/data/images/output/10038_1967300488.png'),
 PosixPath('/home/ubuntu/data/images/output/10038_21429

In [5]:
data = pd.read_csv(train_data)

In [6]:
image_ids = []
for x in train_images:
    filename = x.name
    slug = filename.split("_")[1]
    slug = slug.split(".")[0]
    image_ids.append(int(slug))

In [7]:
images_df = pd.DataFrame()
images_df['image_id'] = image_ids
images_df['image_path'] = train_images

In [8]:
images_df

,image_id,image_path
0,1459541791,/home/ubuntu/data/images/output/10006_14595417...
1,1864590858,/home/ubuntu/data/images/output/10006_18645908...
2,1874946579,/home/ubuntu/data/images/output/10006_18749465...
3,462822612,/home/ubuntu/data/images/output/10006_46282261...
4,1031443799,/home/ubuntu/data/images/output/10011_10314437...
...,...,...
54701,873425464,/home/ubuntu/data/images/output/997_873425464.png
54702,1078943060,/home/ubuntu/data/images/output/9989_107894306...
54703,398038886,/home/ubuntu/data/images/output/9989_398038886...
54704,439796429,/home/ubuntu/data/images/output/9989_439796429...


In [9]:
merged_df = pd.merge(data,images_df,how='inner',on='image_id')

In [10]:
merged_df

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case,image_path
0,2,10006,462822612,L,CC,61.0,0,0,0,NaN,0,NaN,29,False,/home/ubuntu/data/images/output/10006_46282261...
1,2,10006,1459541791,L,MLO,61.0,0,0,0,NaN,0,NaN,29,False,/home/ubuntu/data/images/output/10006_14595417...
2,2,10006,1864590858,R,MLO,61.0,0,0,0,NaN,0,NaN,29,False,/home/ubuntu/data/images/output/10006_18645908...
3,2,10006,1874946579,R,CC,61.0,0,0,0,NaN,0,NaN,29,False,/home/ubuntu/data/images/output/10006_18749465...
4,2,10011,220375232,L,CC,55.0,0,0,0,0.0,0,NaN,21,True,/home/ubuntu/data/images/output/10011_22037523...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54701,1,9973,1729524723,R,MLO,43.0,0,0,0,1.0,0,C,49,False,/home/ubuntu/data/images/output/9973_172952472...
54702,1,9989,63473691,L,MLO,60.0,0,0,0,NaN,0,C,216,False,/home/ubuntu/data/images/output/9989_63473691.png
54703,1,9989,1078943060,L,CC,60.0,0,0,0,NaN,0,C,216,False,/home/ubuntu/data/images/output/9989_107894306...
54704,1,9989,398038886,R,MLO,60.0,0,0,0,0.0,0,C,216,True,/home/ubuntu/data/images/output/9989_398038886...


In [11]:
cancer_df = merged_df['image_path'][merged_df['cancer']==1]
cancer_free_df = merged_df['image_path'][merged_df['cancer']==0]

In [13]:
Config.annotation_directory

PosixPath('/home/ubuntu/data/code/efficientnet-detector/data/dataset/annotations')

In [14]:
for x in cancer_df.sample(n=1000):
    image_path = Path(x)
    slug = image_path.stem
    data = {"image_path":str(x), "label":1}
    annotation_path = Config.annotation_directory /f"{slug}.json"
    with open(annotation_path, "w") as f:
        json.dump(data,f)

In [15]:
for x in cancer_free_df.sample(n=1000):
    image_path = Path(x)
    slug = image_path.stem
    data = {"image_path":str(x), "label":0}
    annotation_path = Config.annotation_directory /f"{slug}.json"
    with open(annotation_path, "w") as f:
        json.dump(data,f)

In [1]:
from efficientnet.dataset import get_sample_dicts, EfficientNetDataset
from efficientnet.utils import Holdout
from efficientnet.config import Config
import torch

In [2]:
Config.annotation_directory

PosixPath('/home/ubuntu/data/code/efficientnet-detector/data/dataset/annotations')

In [3]:
samples = get_sample_dicts()

In [4]:
len(samples)

2000

In [5]:
samples[0]

{'image_path': '/home/ubuntu/data/images/output/10025_562340703.png',
 'image': Array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 'label': array(0., dtype=float32)}

In [6]:
dev_samples = get_sample_dicts(Holdout.test)

In [7]:
len(dev_samples)

197

In [8]:
data = EfficientNetDataset(Holdout.train)

In [9]:
len(data)

1617

In [5]:
dev_data[0]

{'image': tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]),
 'image_path': '/home/ubuntu/data/images/output/10025_562340703.png',
 'label': array(0., dtype=float32)}

In [11]:
loader = torch.utils.data.DataLoader(
            EfficientNetDataset(Holdout.train),
            batch_size=Config.batch_size,
            num_workers=Config.n_workers,
        )

In [12]:
batch = next(iter(loader))

In [13]:
batch['image']

tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

         [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0

In [14]:
batch

{'image': tensor([[[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           ...,
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
 
          [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           ...,
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
 
          [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
           [0.0